In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pylcp 
import scipy.constants  as cts
import magpylib as magpy
from scipy.stats import rv_continuous
from scipy.interpolate import RegularGridInterpolator
import multiprocessing as mp

# magnetic field profile 

In [ ]:
tfmot = magpy.Collection()
N = 0
angles = [0,90,180,270]
angles_d = [45,225,225,45]
l = 5
p = [0,0,0,0,20,20,20,20]
s  = [-1,-1,1,1,1,1,-1,-1]
H = 35
h = [-H,-H,-H,-H,H,H,H,H]
mstyle = dict(
    mode="color+arrow",
    color=dict(north="magenta", middle="white", south="turquoise"),
    arrow=dict(width=2, color="k")
)
for a in range(len(angles)):
    cube= magpy.magnet.Cuboid(
    dimension=(6,8,80),
    polarization=(s[a]*1.42,0,0),
    position=(h[a],h[a],h[a]),
    style_magnetization=mstyle )
    
    cube.rotate_from_angax(angles[a], 'z',anchor = 0)
    cube.rotate_from_angax(angles_d[a], 'z')

    tfmot.add(cube)

magpy.show(tfmot.rotate_from_angax(45,'z'), backend='plotly')

In [ ]:
# Continuation from above - ensure previous code is executed

import matplotlib.pyplot as plt
fig, ax = plt.subplots()

# Compute and plot field on x-y grid
grid = np.mgrid[-10:10:100j, -10:10:100j, 0:0:1j].T[0]
X, Y, _ = np.moveaxis(grid, 2, 0)

B = tfmot.getB(grid)*1e3
Bx, By, _ = np.moveaxis(B, 2, 0)
Bamp = np.linalg.norm(B, axis=2)

pc = ax.contourf(X, Y, Bamp, levels=50, cmap="coolwarm")
ax.streamplot(X, Y, Bx, By, color="k", density=1.5, linewidth=1)

# Add colorbar
fig.colorbar(pc, ax=ax, label="|B| (mT)")

# Figure styling
ax.set(
    xlabel="x-position(mm)",
    ylabel="y-position(mm)",
    aspect=1,
)

plt.show()

In [11]:
x = np.linspace(-5e-3,5e-3,100)
r0  =  5e-3
B0 = 5e-3
y = lambda r: B0*(r/r0)**2


In [ ]:
L = 16
z = np.linspace(-20,20,1000)
b_y = []
b_z = []
b_x = []
b_mag_abs = []
for i in z:
    b_x.append(tfmot.getB([0,i*10,0])[0])
    b_y.append(tfmot.getB([0,i*10,0])[1])
    b_z.append(tfmot.getB([0,i*10,0])[2])
    b_mag_abs.append(np.sqrt((tfmot.getB([0,i*10,0])[0])**2 + (tfmot.getB([0,i*10,0])[1])**2 + (tfmot.getB([0,i*10,0])[2])**2))

fig,ax = plt.subplots(1,1,figsize=(20,5))    
plt.plot(z,np.array(b_x)*1e3)
plt.plot(z,np.array(b_y)*1e3)
plt.plot(z,np.array(b_z)*1e3)
plt.plot(z,np.array(b_mag_abs)*1e3)
plt.plot(x*100,y(x)*1e3)
plt.xlabel('z (cm)')

plt.ylabel('$B$ (mT)')
plt.legend(['$B_y$','$B_y$','$B_z$','mod'])  
plt.ylabel('$B_y$ (mT)')
plt.title('Magnetic field along the z-axis(will going up the same amount in y)')
plt.ylim([-20,20])
plt.grid()



# system setup

In [ ]:
mass_lab = 173*cts.value('atomic mass constant') # YB-173 mass in kg
sd = np.sqrt(cts.k*216.85/mass_lab)
klab = 2*np.pi*25068.2222# Lab wavevector (without 2pi) in cm^{-1}   # Lifetime of 6P_{3/2} state (in seconds)
gammalab = (1.93*1e8)
Blab = -67*1e-3#(v0*cts.hbar*(klab*100))/cts.value('Bohr magneton')
b_0L =  48*1e-3
# (v0*cts.hbar*(klab*100))/cts.value('Bohr magneton')/2
# T # About 15 G/cm is a typical gradient for Rb
L = 16
x0 = 1/klab  # cm
t0 = 1/gammalab  # s
#100*(mass_lab*(v0**2))/(0.5*cts.hbar*(klab*100)*gammalab) +5
x0 = 1/klab  # cm
t0 = 1/gammalab  # s
vc = x0/t0*1/(100)
Fc = gammalab*cts.hbar*klab*100

print(Blab, L,b_0L)

In [ ]:
x0 = 1/klab  # cm
t0 = 1/gammalab  # s

mass = mass_lab*(x0*1e-2)**2/cts.hbar/t0

# And now our wavevector, decay rate, and magnetic field gradient in these units:
k = klab*x0
gamma = gammalab*t0
alpha = cts.value('Bohr magneton')*Blab*t0/cts.hbar
b_0 = cts.value('Bohr magneton')*b_0L*t0/cts.hbar 
l = L/x0 

print(x0, t0, mass, k, gamma, alpha,b_0)

In [ ]:
gammalab/(2*np.pi)*1e-6

## hamiltonian setup and lasers

In [16]:
# Define the atomic Hamiltonian:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6,Bhfs = 601.87*1e6 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



#ham = pylcp.hamiltonian(Hg, He, mugq, mueq, dijq, mass=mass, gamma=gamma, k=k)
# Define the atomic Hamiltonian:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
0, 5/2, 0,  	-0.2592,
    Ahfs = 0, Bhfs=0, Chfs=0,
    muB=1)# ground state 1s0
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
1, 5/2, 1.035,	-0.2592,
    Ahfs=59.52*1e6*t0,Bhfs = 601.87*1e6*t0 , Chfs= 0,
    muB=1) #excited state 1p1

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(0, 1, 5/2)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))
hamiltonian_D2 = pylcp.hamiltonian(mass = mass)
hamiltonian_D2.add_H_0_block('g', H_g_D2)
hamiltonian_D2.add_H_0_block('e', H_e_D2)
hamiltonian_D2.add_d_q_block('g', 'e', dijq_D2, gamma = gamma, k = k)
hamiltonian_D2.add_mu_q_block('g', mu_q_g_D2)
hamiltonian_D2.add_mu_q_block('e', mu_q_e_D2)



#ham = pylcp.hamiltonian(Hg, He, mugq, mueq, dijq, mass=mass, gamma=gamma, k=k)


s = 0
det = 0
r = 0.4/x0 
# this is for cylindrcal laser howver we want an eliptical one as this for somereason turns out to be better 
# Define the laser beams:
laserBeams = pylcp.laserBeams([
    { 'kvec':np.array([0, 1, 0]),  's': s,'pol_coord':'spherical', 'pol':1, 'delta': (E_e_D2[2]-E_g_D2[0]) + det*gamma},
    { 'kvec':np.array([0, -1, 0]), 's': s,'pol_coord':'spherical', 'pol':1, 'delta': (E_e_D2[2]-E_g_D2[0]) + det*gamma},
    {'kvec':np.array([1, 0, 0]), 's': s,'pol_coord':'spherical', 'pol':-1, 'delta':  (E_e_D2[2]-E_g_D2[0]) + det*gamma},
    {'kvec':np.array([-1, 0, 0]), 's': s,'pol_coord':'spherical', 'pol':-1, 'delta': (E_e_D2[2]-E_g_D2[0]) + det*gamma}
     ],
    beam_type=pylcp.infinitePlaneWaveBeam) 

#linGrad =  pylcp.magField(lambda R: mag(R,b_0,alpha,l))
alpha = cts.value('Bohr magneton')*0.5*t0/cts.hbar
bf_mot = lambda R:tfmot.getB(R*10*x0)*cts.value('Bohr magneton')*t0/cts.hbar 
bf2_mot = lambda R: -alpha*R
linGrad = pylcp.magField(lambda R: bf_mot(R))

In [17]:
rateeq = pylcp.rateeq(laserBeams, linGrad, hamiltonian_D2, include_mag_forces=False)

# force profile caculations 

In [ ]:
position = np.linspace(-0.4,0.4,20)
max_force = []
#for i in position:    

Fc = cts.hbar*klab*gammalab
x = np.arange(-5, 5, 0.05)/x0
y = np.linspace(0,0,1)/x0

v = np.arange(-25, 25, 0.5)/vc

V,Y = np.meshgrid(v,y)

Rvec = np.array([np.zeros(Y.shape), np.zeros(Y.shape), np.zeros(Y.shape)])
Vvec = np.array([np.zeros(V.shape), V, np.zeros(V.shape)])
rateeq.generate_force_profile(Rvec, Vvec, name='Fz', progress_bar=True)


In [ ]:
position = np.linspace(-0.4,0.4,20)
max_force = []
#for i in position:    

Fc = cts.hbar*klab*gammalab
x = np.arange(-5, 5, 0.05)/x0
y = np.linspace(-4,4,100)/x0

v = np.linspace(0,0 ,1)/vc

V,Y = np.meshgrid(v,y)

Rvec = np.array([np.zeros(Y.shape), Y, np.zeros(Y.shape)])
Vvec = np.array([np.zeros(V.shape), V, np.zeros(V.shape)])
rateeq.generate_force_profile(Rvec, Vvec, name='Fx', progress_bar=True)

In [20]:
# fig,ax = plt.subplots(1,1,figsize=(20,5))
# Fc = gammalab*cts.hbar*klab*100
# im1 = ax.contourf(np.array(v)*vc, np.array(y)*x0,rateeq.profile['Fz'].F[1]*Fc)
# fig.subplots_adjust(left=0.08, wspace=0.2)
# cb1 = plt.colorbar(im1)
# cb1.set_label('$F(N)$')
# ax.set_ylabel('$y$ (cm)')
# ax.set_xlabel('$v (ms^{-1})$')
# plt.title('Contour plot of position vs velocity and force')
# plt.xlim([-14,14])

In [ ]:
rateeq.profile['Fx'].F[1][:,0]*Fc

In [ ]:
plt.plot(y*x0,rateeq.profile['Fx'].F[1][:,0]*Fc)

In [794]:
# Constants
k_B = cts.k  # Boltzmann constant in J/K
 # Atomic mass of Yb-173
mass_kg = mass_lab  # Convert to kilograms
temperature = 405 + 273.15  # Convert Celsius to Kelvin

# Velocity Probability Function
def p(m, T, v):
    return (m / (k_B * T))**2 * np.exp(-m * v**2 / (2 * k_B * T)) * (v**3) / 2

In [795]:
class MaxwellBoltzmann(rv_continuous):
    def _pdf(self, v):
        return p(mass_kg, temperature, v)

# Define the custom distribution
maxwell_boltzmann = MaxwellBoltzmann(a=0, name="Maxwell-Boltzmann")

In [ ]:
size = 100# Number of samples
velocities = maxwell_boltzmann.rvs(size=size)
velocities = velocities[velocities < 400]
velocities_t = [np.tan(35e-3)*i for i in velocities ]
v_0 = []
for i in range(len(velocities)):
    v_0.append([velocities_t[i],velocities_t[i],50])
# Display the sampled velocities
print(len(velocities))
plt.hist(velocities, bins=100, density=True)
plt.grid()


In [797]:
def simulate_trajectory_zeeman(v_0, t_span,mass_lab):
    x_t,y_t, v_tx, v_ty, t = [], [], [],[], []
    di = 0  # Initial conditions
    s = [0,0,0]
    for q in range(len(t_span)):
        v_x = v_0[0]

        v_y = v_0[1]

        v_z = v_0[2]
    
        dt = t_span[q] - di
        
        f  = 0

        # Interpolate the force profile

        # Update position and velocity
   
        a_x   = 0
        a_y   = 0
        a_z   = 0

    

        s_x = (1/2)*a_x*(dt**2) + v_x*dt + s[0]
        
        s_y = (1/2)*a_y*(dt**2) + v_y*dt + s[0]

        s_z = (1/2)*a_z*(dt**2) + v_z*dt + s[0]
        
        v_x = v_x + a_x*dt

        v_y = v_y + a_y*dt

        v_z = v_z + a_z*dt

        # Append values
        x_t.append(s_x)
        y_t.append(s_y)
        v_tx.append(v_x)
        v_ty.append(v_y)
        t.append(t_span[q])
        di = t_span[q]
        v_0 = (v_x,v_y,v_z)
        s = (s_x,s_y,s_z)
 

    return x_t, y_t, v_tx, v_ty, t

In [ ]:
rateeq.generate_force_profile(np.array([0.3318885197521952, 0.3318885197521952, 4.080525528605002]),np.array([0,0,0]), name='Fz', progress_bar=False)
print(rateeq.profile['Fz'].F[1])

In [ ]:
x0_m = x0 / 100

def simulate_trajectory(v_0, t_span, mass_lab):

    x_t, y_t, v_tx, v_ty, t = [], [], [], [], []

    di = 0  # Initial conditions

    s = np.array([0,0,0])
    
    b = 0

    for q in range(len(t_span)):

        v_x = v_0[0]

        v_y = v_0[1]

        v_z = v_0[2]


        #print(v_0)
        dt = (t_span[q] - di)/t0
        
        # if np.isnan(v_0).any():
        #     print(v_0 , s,dt,a_x)
        #     break

        b  = b +1

        #if any(v_0) < 1e-10:
         #   print(v_0)
          #  break
        
        if np.isnan(v_0).any():
            print(f'final b is {b}') 
            break
    
  
  
        rateeq.generate_force_profile(np.array(v_0), np.array(s), name='Fz', progress_bar=False)
     
        
        # Update position and velocity

        a_x = rateeq.profile['Fz'].F[0]/mass
        
        a_y = rateeq.profile['Fz'].F[1]/mass

        a_z = 0 

        s_x = (1 / 2) * a_x * (dt ** 2) + v_x * dt + s[0]

        s_y = (1 / 2) * a_y * (dt ** 2) + v_y * dt + s[1]

        s_z = (1 / 2) * a_z * (dt ** 2) + v_z * dt + s[2]
        #print(a_y)

        v_x = v_x + a_x * dt

        v_y = v_y + a_y * dt

        v_z = v_z + a_z * dt
        #print(v_x)
        # Append values
        x_t.append(s_x)
        y_t.append(s_y)
        v_tx.append(v_x)
        v_ty.append(v_y)
        t.append(t_span[q])

        di = t_span[q]
        v_0 = [v_x, v_y, v_z]
        s = [s_x, s_y, s_z]
        # print('sped')
        # print(v_0)
        # print('dis')
        # print(s)
        #print(dt)
    return x_t, y_t, v_tx, v_ty, t

In [ ]:
# The 2D MOT seems to be slowing although the function is still tempramental working only for some values of v_0 which 
# suggest to me the units are not correct. howver i can not seem to see where the units are wrong.
x_n = []
y_n = []
v_tx_n = []
v_ty_n = []
t_n = []
t_max = 0.05
t_span = np.linspace(0, t_max, 1000)
v_01 = [v_0[0]]
b = 0
for i in v_01:
    x_t, y_t, v_tx, v_ty, t = simulate_trajectory(np.array(i)/vc, t_span, mass_lab)

    x_n.append(x_t)
    y_n.append(y_t)
    v_tx_n.append(v_tx)
    v_ty_n.append(v_ty)
    t_n.append(t)

    b += 1
    print(b)
    

In [ ]:
rateeq.generate_force_profile([0.33187919969227325, 0.33187918493053986, 4.080525528605002],[35264.28422765166, 35264.28273070033, 433581.3662276487],name= 'Fz')

print(rateeq.profile['Fz'].F[2])
print(np.array([0.33187919969227325, 0.33187918493053986, 4.080525528605002])*vc,np.array([35264.28422765166, 35264.28273070033, 433581.3662276487])*x0)


In [ ]:

# t_max = 0.005
# t_span =  np.linspace(0, t_max, 1000)
# theta = 37e-3
# # Define a list of initial velocities (v_0)
# initial_velocities = v_0

# # Create a pool of processes
# with mp.Pool(processes = mp.cpu_count()) as pool:
#     # Map only v_0 to simulate_trajectory; pass constant args using `starmap`
#     results = pool.starmap(
#         simulate_trajectory, [(v, t_span, mass_lab) for v in initial_velocities]
#     )

# # Save results into a new list
# all_trajectories = []
# for result in results:
#     all_trajectories.append(result)

# # Print the saved list
# for i, trajectory in enumerate(all_trajectories):
#     print(f"Trajectory {i + 1}:")
#     print("x_t:", trajectory[0])
#     print("y_t:", trajectory[1])
#     print()

In [ ]:
initial_y = []
final_y = []
for i in range(len(x_n)):

    final_y.append(v_ty_n[i][-1]*vc)
    initial_y.append(v_ty_n[i][0]*vc)

In [ ]:
k  = np.linspace(2.5,14,100)

plt.scatter(initial_y,final_y)
plt.xlabel('Initial velocity(m/s)')
plt.ylabel('Final velocity(m/s)')
plt.plot(k,k,'r')
plt.legend(['MOT simulation','y=x'])